## Initialize the data-generator

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf

from google.protobuf import text_format
from modeling import trainer
from protos import model_pb2
from protos import pipeline_pb2
from readers import reader


model_dir = "logs/baseline/"
video_lengths_csv = "epic-kitchens-100-annotations/EPIC_100_video_info.csv"
verb_classes_csv = "epic-kitchens-100-annotations/EPIC_100_verb_classes.csv"
noun_classes_csv = "epic-kitchens-100-annotations/EPIC_100_noun_classes.csv"


for gpu in tf.config.experimental.list_physical_devices('GPU'):
  tf.config.experimental.set_memory_growth(gpu, True)

with tf.io.gfile.GFile(os.path.join(model_dir, 'pipeline.pbtxt'), 'r') as fp:
  pipeline_proto = text_format.Merge(fp.read(), pipeline_pb2.Pipeline())

# HERE WE VISUALIZE TRAIN SET PREDICTIONS.
input_fn = reader.get_input_fn(pipeline_proto.train_reader, is_training=False)
model_fn = trainer.create_model_fn(pipeline_proto)

features, labels = input_fn().make_one_shot_iterator().get_next()
predictions = model_fn(features, labels, tf.estimator.ModeKeys.PREDICT, None).predictions

def data_generator():
  saver = tf.train.Saver()
  with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(model_dir)
    saver.restore(sess, ckpt.model_checkpoint_path)
    while True:
      yield sess.run([predictions, labels])

dg = data_generator()

W0906 23:14:43.983753 140435088435008 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


ModuleNotFoundError: No module named 'joblib'

## Visualize the per-segment prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from matplotlib import colors
from scipy.special import softmax

STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque', 'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan', 'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet', 'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod', 'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue', 'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue', 'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid', 'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin', 'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed', 'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown', 'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow', 'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat',
    'WhiteSmoke', 'Yellow', 'YellowGreen'
]
random.seed(286)
random.shuffle(STANDARD_COLORS)
STANDARD_COLORS = ['White'] + STANDARD_COLORS


def show_color_bar(data, classid2colorid, classid2name, title, trim_to=100, show_ticks=True):
  observation_window = data.shape[0]
  image = np.full((10, observation_window, 3), 255)
  for i in range(observation_window):
    color_id = classid2colorid[data[i]]
    r, g, b = colors.to_rgb(STANDARD_COLORS[color_id].lower())
    image[:, i, 0] = int(r * 255)
    image[:, i, 1] = int(g * 255)
    image[:, i, 2] = int(b * 255)
    
  plt.figure(figsize=(20, 2))
  ax = plt.subplot(111)
  ax.matshow(image[:, :trim_to, :])
  if show_ticks:
    ax.set_xticks(np.arange(trim_to))
    ax.set_xticklabels([classid2name.get(x, '') + '({})'.format(x) for x in data[:trim_to]])
  plt.title(title)
  plt.xticks(rotation=90)

    
def load_id_to_name(file_name):
  df = pd.read_csv(file_name)
  return {i + 1: v for i, v in zip(df['id'], df['key'])}


classid2verb = load_id_to_name(verb_classes_csv)
classid2noun = load_id_to_name(noun_classes_csv)

elem = next(dg)
y_pred, y_true = elem
y_pred_noun, y_pred_verb = y_pred['noun_logits'][0].argmax(-1), y_pred['verb_logits'][0].argmax(-1)
y_true_noun, y_true_verb = y_true[0][0], y_true[1][0]
observation_window = y_true_verb.shape[0]

verbid2colorid = {c: i for i, c in enumerate(sorted(set(y_true_verb.tolist() + y_pred_verb.tolist())))}
show_color_bar(y_true_verb, verbid2colorid, classid2verb, "verb-true")
show_color_bar(y_pred_verb, verbid2colorid, classid2verb, "verb-pred")

nounid2colorid = {c: i for i, c in enumerate(sorted(set(y_true_noun.tolist() + y_pred_noun.tolist())))}
show_color_bar(y_true_noun, nounid2colorid, classid2noun, "noun-true")
show_color_bar(y_pred_noun, nounid2colorid, classid2noun, "noun-pred")

## Visualize the confidense score of specific classes

In [ ]:
noun_scores, verb_scores = softmax(y_pred['noun_logits'][0], -1), softmax(y_pred['verb_logits'][0], -1)

plt.figure(figsize=(20, 4))
plt.bar(np.arange(100), verb_scores[:100, 3])

## Post-processing

In [ ]:
from models.post_process import py_post_process

def load_video_lengths(file_name):
  df = pd.read_csv(file_name, sep=',')
  return {i: l for i, l in zip(df['video_id'], df['duration'])}

trim_to = 100
max_n_detection = 100
video_lengths = load_video_lengths(video_lengths_csv)

(i_starts, i_ends, verb_ids, confidences
 ) = py_post_process(verb_scores[:trim_to, 1:], 
                     max_n_detection=max_n_detection, 
                     score_min=0.1, 
                     score_max=0.1, 
                     score_step=0.2)

show_color_bar(y_true_verb, verbid2colorid, classid2verb, "verb-true", show_ticks=False)

images = []
for i_start, i_end, verb_id, confidence in zip(i_starts, i_ends, verb_ids, confidences):
  verb_id += 1
  if not verb_id in verbid2colorid: continue
        
  color_id = verbid2colorid[verb_id]
  image = np.full((1, trim_to, 3), 255)
  r, g, b = colors.to_rgb(STANDARD_COLORS[color_id].lower())
  for i in range(i_start, 1 + i_end):
    image[:, i, 0] = int(r * 255)
    image[:, i, 1] = int(g * 255)
    image[:, i, 2] = int(b * 255)
  images.append(image)

image = np.concatenate(images, 0)
plt.figure(figsize=(20, 200))
ax = plt.subplot(111)
ax.matshow(image[:, :, :])